In [9]:
import pandas as pd
from quant_utils.db_conn import DB_CONN_JJTG_DATA
from quant_utils.constant import DATE_FORMAT
import os
import quant_utils. data_moudle as dm
from quant_utils.constant_varialbles import  LAST_TRADE_DT
from fund_db.fund_performance import BasePerformance
import datetime
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
from quant_utils.utils import make_dirs

In [10]:
file_path = 'C:/Users/Wilcoxon/Desktop/配置excel/投顾大赛净值/'
csv_list = os.listdir(file_path)
name_dict = {
    'ROBOT_NAME': 'PORTFOLIO_NAME',
    'D_DATE': "TRADE_DT",
    'VALUE': 'PORTFOLIO_RET_ACCUMULATED',
}

In [ ]:
def get_tg_portfolio_info():
    query = """
    SELECT 
    a.*, 
    b.PEER_QUERY,
    b.PEER_PORTFOLIO
    FROM tg_portfolio_info a 
    join tg_peer b on a.SECTOR = b.SECTOR
    """
    return DB_CONN_JJTG_DATA.exec_query(query)
tg_portfolio_info = get_tg_portfolio_info()

In [ ]:
for csv in csv_list[-1:]:
    csv_path = os.path.join(file_path, csv)
    df = (
        pd.read_csv(csv_path)
        .query("委托类型 == '成立以来累计收益率'")
    )[["ROBOT_NAME", "D_DATE", "VALUE"]]
    df.rename(columns=name_dict, inplace=True)
    df["TRADE_DT"] = pd.to_datetime(df["TRADE_DT"])
    df["TRADE_DT"] = df["TRADE_DT"].dt.strftime(DATE_FORMAT)
    df = df[df["PORTFOLIO_NAME"].isin(tg_portfolio_info["PORTFOLIO_NAME"])]
    df["PORTFOLIO_RET_ACCUMULATED"] = df["PORTFOLIO_RET_ACCUMULATED"] * 100
    DB_CONN_JJTG_DATA.upsert(df, table="tg_portfolio_ret")

In [13]:
def get_tg_rank(portfolio_name: str, end_date: str) -> pd.DataFrame:
    peer_query = (
        get_tg_portfolio_info()
        .query(f"PORTFOLIO_NAME == '{portfolio_name}'")["PEER_QUERY"]
        .values[0]
    )
    peer_query = peer_query.replace("LEVEL", "c.LEVEL")
    peer_query = peer_query.replace("==", "=")
    query_sql = f"""
        WITH a AS (
            SELECT
                b.DATE_NAME,
                a.TICKER_SYMBOL,
                a.START_DATE,
                a.END_DATE,
                a.CUM_RETURN,
                a.MAXDD 
            FROM
                tg_performance_inner a
                JOIN portfolio_dates b ON a.START_DATE = b.START_DATE 
                AND a.END_DATE = b.END_DATE
            WHERE
                1 = 1 
                AND a.END_DATE = '{end_date}' 
                AND b.DATE_NAME = 'QTD' 
                AND a.TICKER_SYMBOL = '{portfolio_name}' UNION
            SELECT
                b.DATE_NAME,
                a.TICKER_SYMBOL,
                a.START_DATE,
                a.END_DATE,
                a.CUM_RETURN,
                a.MAXDD 
            FROM
                fund_performance_inner a
                JOIN portfolio_dates b ON a.START_DATE = b.START_DATE 
                AND a.END_DATE = b.END_DATE 
                JOIN fund_type_own c ON c.TICKER_SYMBOL = a.TICKER_SYMBOL
            WHERE
                1 = 1 
                and {peer_query}
                AND a.END_DATE = '{end_date}' 
                AND b.DATE_NAME = 'QTD' 
            AND ( 
                c.REPORT_DATE = ( 
                    SELECT max( report_date ) 
                    FROM fund_type_own 
                    WHERE PUBLISH_DATE <= '{end_date}' 
                )
                )),
            b AS ( SELECT DATE_NAME, count(*) AS NUM FROM a GROUP BY DATE_NAME ) SELECT
            a.DATE_NAME,
            a.TICKER_SYMBOL,
            a.START_DATE,
            a.END_DATE,
            '累计收益率' AS INDICATOR,
            CUM_RETURN AS PORTFOLIO_VALUE,
            CONCAT( ROW_NUMBER() over ( PARTITION BY a.DATE_NAME ORDER BY CUM_RETURN DESC ), '/', b.NUM ) AS PEER_RANK,
            PERCENT_RANK() over ( PARTITION BY a.DATE_NAME ORDER BY CUM_RETURN DESC )* 100 AS PEER_RANK_PCT 
        FROM
            a
            JOIN b ON b.DATE_NAME = a.DATE_NAME  UNION
        SELECT
            a.DATE_NAME,
            a.TICKER_SYMBOL,
            a.START_DATE,
            a.END_DATE,
            '最大回撤' AS INDICATOR,
            MAXDD AS PORTFOLIO_VALUE,
            CONCAT( ROW_NUMBER() over ( PARTITION BY a.DATE_NAME ORDER BY MAXDD ), '/', b.NUM ) AS PEER_RANK,
            PERCENT_RANK() over ( PARTITION BY a.DATE_NAME ORDER BY MAXDD )* 100 AS PEER_RANK_PCT 
        FROM
            a
            JOIN b ON b.DATE_NAME = a.DATE_NAME 
    """
    df = DB_CONN_JJTG_DATA.exec_query(query_sql).query(
        f"TICKER_SYMBOL == '{portfolio_name}'"
    )
    benchmark_portfolio = (
        get_tg_portfolio_info()
        .query(f"PORTFOLIO_NAME == '{portfolio_name}'")["PEER_PORTFOLIO"]
        .values[0]
    )
    benmark_sql = f"""
    SELECT
        a.START_DATE,
        a.END_DATE,
        '累计收益率' as INDICATOR,
        a.CUM_RETURN AS BENCHMARK_VALUE_INNER 
    FROM
        benchmark_performance_inner a
        JOIN portfolio_dates b ON a.START_DATE = b.START_DATE 
        AND a.END_DATE = b.END_DATE 
    WHERE
        1 = 1 
        AND a.`TICKER_SYMBOL` = '{benchmark_portfolio}' 
        AND b.DATE_NAME = 'QTD' 
        AND a.END_DATE = '{end_date}' UNION
        select
        a.START_DATE,
        a.END_DATE,
        '最大回撤' as INDICATOR,
        a.MAXDD AS BENCHMARK_VALUE_INNER 
    FROM
        benchmark_performance_inner a
        JOIN portfolio_dates b ON a.START_DATE = b.START_DATE 
        AND a.END_DATE = b.END_DATE 
    WHERE
        1 = 1 
        AND a.`TICKER_SYMBOL` = '{benchmark_portfolio}' 
        AND b.DATE_NAME = 'QTD' 
        AND a.END_DATE = '{end_date}'
    """
    benchmark_df = DB_CONN_JJTG_DATA.exec_query(benmark_sql)

    return df.merge(benchmark_df, how="left", on=["START_DATE", "END_DATE", "INDICATOR"])

In [14]:
def update_tg_portfolio_performance(
    start_date: str, end_date: str, portfolio_name_list: list = None
) -> None:
    """
    更新组合表现及衍生组合表现

    Parameters
    ----------
    start_date : str
        开始日期
    end_date : str
        结束日期
    """
    trade_dates = dm.get_period_end_date(
        start_date=start_date, end_date=end_date, period="d"
    )
    portfolio_info = get_tg_portfolio_info()
    portfolio_info["LISTED_DATE"] = portfolio_info["LISTED_DATE"].apply(
        lambda x: x.strftime(DATE_FORMAT)
    )

    for date in tqdm(trade_dates):
        # 写入自己计算的组合
        portfolio_names = portfolio_info.query(f"LISTED_DATE <= '{date}'")[
            "PORTFOLIO_NAME"
        ].tolist()

        if portfolio_name_list is not None:
            portfolio_names = list(set(portfolio_names) & set(portfolio_name_list))

        result_list = [get_tg_rank(i, date) for i in tqdm(portfolio_names)]
        # result_list = Parallel(n_jobs=-1)(
        #     delayed(get_tg_rank)(portfolio_name, date) for portfolio_name in tqdm(portfolio_names)
        # )
        # result_list = [
        #     get_tg_rank(portfolio_name, date)
        #     for portfolio_name in tqdm(portfolio_names)
        # ]
        result_df = pd.concat(result_list)
        result_df.rename(columns={"DATE_NAME": "CYCLE"}, inplace=True)
        DB_CONN_JJTG_DATA.upsert(result_df, "tg_portfolio_performance")


In [15]:
class TGPerformance(BasePerformance):
    def get_nav(self):
        query_sql = f"""
        SELECT
            DATE_FORMAT( TRADE_DT, "%Y%m%d" ) AS END_DATE,
            a.PORTFOLIO_NAME AS TICKER_SYMBOL,
            (portfolio_ret_ACCUMULATED/100 + 1) AS NAV 
        FROM
            tg_portfolio_ret a
            JOIN tg_portfolio_info b ON a.PORTFOLIO_NAME = b.PORTFOLIO_NAME 
        WHERE
            1 = 1 
            AND a.TRADE_DT >= b.LISTED_DATE
            AND a.TRADE_DT <= '{self.end_date}'
        """
        df = DB_CONN_JJTG_DATA.exec_query(query_sql)
        return df

In [ ]:
today = datetime.datetime.now().strftime("%Y%m%d")
start_date = dm.offset_trade_dt(LAST_TRADE_DT, 2)
end_date = LAST_TRADE_DT
tg = TGPerformance(start_date=start_date, end_date=end_date)
tg.write_data("tg_performance_inner")
update_tg_portfolio_performance(start_date=start_date, end_date=end_date)

In [17]:
def get_tg_evaluation_result(end_date):
    query = f"""
        WITH a AS (
            SELECT
                END_DATE,
                TICKER_SYMBOL,
                sum( CASE INDICATOR WHEN '累计收益率' THEN PORTFOLIO_VALUE ELSE 0 END ) AS CUM_RET,
                sum( CASE INDICATOR WHEN '最大回撤' THEN PORTFOLIO_VALUE ELSE 0 END ) AS MAXDD,
                sum( CASE INDICATOR WHEN '累计收益率' THEN 100 - PEER_RANK_PCT ELSE 0 END ) AS CUM_RET_SCORE,
                sum( CASE INDICATOR WHEN '最大回撤' THEN 100 - PEER_RANK_PCT ELSE 0 END ) AS MAXDD_SCORE,
                sum( CASE INDICATOR WHEN '累计收益率' THEN BENCHMARK_VALUE_INNER ELSE 0 END ) AS CUM_RET_BENCHMARK,
                sum( CASE INDICATOR WHEN '最大回撤' THEN BENCHMARK_VALUE_INNER ELSE 0 END ) AS MAXDD_BENCHMARK 
            FROM
                `tg_portfolio_performance` 
            WHERE
                1 = 1 
                AND END_DATE = '{end_date}' 
                and cycle = 'QTD'
            GROUP BY
                END_DATE,
                TICKER_SYMBOL 
            ),
            e AS ( SELECT END_DATE, TICKER_SYMBOL, PEER_RANK AS '收益排名' FROM `tg_portfolio_performance` WHERE 1 = 1 AND END_DATE = '{end_date}' AND INDICATOR = '累计收益率'  and cycle = 'QTD'),
            f AS ( SELECT END_DATE, TICKER_SYMBOL, PEER_RANK AS '最大回撤排名' FROM `tg_portfolio_performance` WHERE 1 = 1 AND END_DATE = '{end_date}' AND INDICATOR = '最大回撤'  and cycle = 'QTD'),
            b AS (
            SELECT
                a.*,
                a.CUM_RET_SCORE * 0.75 + a.MAXDD_SCORE * 0.25 AS TOTAL_SCORE,
                b.SECTOR,
                b.SUB_COMPANY,
                b.MANAGER 
            FROM
                a
                JOIN tg_portfolio_info b ON a.TICKER_SYMBOL = b.PORTFOLIO_NAME 
            ),
            c AS (
            SELECT
                b.END_DATE AS '结束日期',
                b.SECTOR AS '赛区',
                b.TICKER_SYMBOL AS '组合名称',
                b.SUB_COMPANY AS '分公司',
                b.MANAGER AS '主理人',
                b.CUM_RET AS '累计收益率%',
                b.MAXDD AS '最大回撤%',
                b.CUM_RET_SCORE AS '累计收益排名%',
                b.MAXDD_SCORE AS '最大回撤排名%',
                b.TOTAL_SCORE AS '总分',
                b.CUM_RET_BENCHMARK,
                b.MAXDD_BENCHMARK,
                RANK() over ( PARTITION BY SECTOR ORDER BY TOTAL_SCORE DESC, CUM_RET DESC ) AS '赛区排名' 
            FROM
                b 
            ),
            d AS ( SELECT `赛区`, count(*) AS '数量' FROM c GROUP BY `赛区` ),
            g AS (
            SELECT
                c.`结束日期` AS '结束日期',
                c.`赛区` AS '赛区',
                c.`组合名称` AS '组合名称',
                c.`分公司` AS '分公司',
                c.`主理人` AS '主理人',
                c.`累计收益率%` AS '累计收益率%',
                e.`收益排名`,
                c.`累计收益排名%` AS '累计收益排名%',
                c.CUM_RET_BENCHMARK AS '累计收益比较基准',
                c.`最大回撤%` AS '最大回撤%',
                f.`最大回撤排名`,
                c.`最大回撤排名%` AS '最大回撤排名%',
                c.MAXDD_BENCHMARK AS '最大回撤比较基准',
                c.`总分` AS '总分',
                CONCAT( c.`赛区排名`, '/', d.`数量` ) AS '赛区排名',
                c.`赛区排名` as `赛区排名_2`
            FROM
                c
                JOIN d ON c.`赛区` = d.`赛区`
                JOIN e ON e.END_DATE = c.`结束日期` 
                AND e.TICKER_SYMBOL = c.`组合名称`
                JOIN f ON f.END_DATE = c.`结束日期` 
                AND f.TICKER_SYMBOL = c.`组合名称` 
            ORDER BY
                `赛区`,
                `总分` DESC,
                `累计收益率%` DESC 
            ) SELECT
            g.*,
        CASE
                WHEN g.`累计收益率%` >= g.`累计收益比较基准` THEN
                '是' ELSE '否' 
            END AS '收益战胜基准',
        CASE
                WHEN g.`最大回撤%` <= g.`最大回撤比较基准` THEN
                '是' ELSE '否' 
            END AS '回撤战胜基准' 
        FROM
            g
    """
    return DB_CONN_JJTG_DATA.exec_query(query)



In [ ]:
start_date = dm.offset_trade_dt(LAST_TRADE_DT, 2)
end_date = dm.offset_trade_dt(LAST_TRADE_DT,1)

dates = dm.get_trade_cal(start_date=start_date, end_date=end_date)
file_path = 'D:/投顾大赛/'

for date in dates:
    df = get_tg_evaluation_result(date)
    make_dirs(file_path)
    turnover = (
        pd.read_csv(csv_path)
        .query("委托类型 == '成立以来换手率'")
    )[["ROBOT_NAME", "VALUE"]]
    turnover.rename(
        columns={"VALUE": "TURNOVER", "ROBOT_NAME": "组合名称", "D_DATE": "结束日期"},
        inplace=True
    )
    df = pd.merge(df, turnover, on=["组合名称"], how='left')
    df.to_excel(os.path.join(file_path, f'大赛榜单{date}.xlsx'), index=False)